In [1]:
import importlib

import ase.io as aio

import os
import numpy as np
import qml
import ase
import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import prepare_calculations as pc
importlib.reload(pc)

import explore_qml_data as eqd
import utils_qm as uqm
import glob
import cpmd_io
importlib.reload(cpmd_io)
import shutil
import collections

In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor rele

In [2]:
def parse_xyz_for_CPMD_input(path2xyz):
    # get structure information from xyz file
    molecule = aio.read(path2xyz)

    atom_symbolsEl = []
    atom_symbolsIdx = []
    for i, el in enumerate(molecule.get_chemical_symbols()):
        atom_symbolsEl.append(el)
        atom_symbolsIdx.append(el + str(i+1))
    atom_symbols = {'el':atom_symbolsEl, 'elIdx':atom_symbolsIdx}
    nuc_charges = molecule.get_atomic_numbers()
    valence_charges = eqd.get_val_charges(nuc_charges)
    positions = molecule.get_positions()
    return(atom_symbols, nuc_charges, positions, valence_charges)

def check_correct_charge():
    compounds = ['CH4', 'H2O', 'NH3', 'HF', 'ethane', 'ethene', 'ethyne', 'HCN']

    lambda_values = np.arange(0.05, 1.05, 0.05)#np.linspace(0, 0.9, 10)
    for comp in compounds:
        for lam in lambda_values:

            # path to PP-files and type of PP (pbe)
            pp_dir = '/data/sahre/PP_LIBRARY'
            pp_type = 'GH_PBE'

            # input templates
            template_inp = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template.inp'
            template_inp_small_lambda = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_small_lambda.inp'

            # read optimized geometry and extract molecule specific parameters
            xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
            atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
            # define the array of lambda values
            lambda_value = np.full_like(valence_charges, fill_value=lam, dtype='float')
            # calculate correct boxsize
            num_ve = valence_charges.sum()
            reference_density = 8/(15**3) # density of UEG for a boxlength of 15 Ang and 8 valence electrons
            boxsize = (num_ve/reference_density)**(1/3)

            shift2center = True

            path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/lam_{np.round(lam, 3)}'

            pp_files = glob.glob(path+'/*_GH_PBE')
            num_ve = 0
            for p in pp_files:
                with open(p, 'r') as f:
                    for line in f:
                        if 'ZV' in line:
                            num_ve += float(line.split()[-1])

            # if np.abs(np.floor(num_ve) +0.5 - num_ve) < 0.1:
            #     print(path)
            num_ve = np.round(num_ve,0)

            with open(path+'/run.inp', 'r') as f:
                for line in f:
                    if 'CHARGE' in line:
                        charge = float(f.readline().strip('\n'))
            if (-charge + num_ve) - valence_charges.sum() > 1e-10:
                print(path)

# debug wrong number of electrons

In [112]:
compounds = ["HCN"]#['CH4', 'H2O', 'NH3', 'HF', 'ethane', 'ethene', 'ethyne', 'HCN']
paths_tar=[]
lambda_values = np.array([0.05, 0.25, 0.45, 0.65, 0.85])
for comp in compounds:
    for lam in lambda_values:
        
        # path to PP-files and type of PP (pbe)
        pp_dir = '/data/sahre/PP_LIBRARY'
        pp_type = 'GH_PBE'
        
        # input templates
        template_inp = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template.inp'
        template_inp_small_lambda = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_small_lambda.inp'
        
        # read optimized geometry and extract molecule specific parameters
        xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
        atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
        # define the array of lambda values
        lambda_value = np.full_like(valence_charges, fill_value=lam, dtype='float')
        # calculate correct boxsize
        num_ve = valence_charges.sum()
        reference_density = 8/(15**3) # density of UEG for a boxlength of 15 Ang and 8 valence electrons
        boxsize = (num_ve/reference_density)**(1/3)

        shift2center = True

        path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/debug/lam_{np.round(lam, 3)}'
        
        # paths_tar.append(f'./{comp}/lam_{np.round(lam, 3)}\n')
        pc.create_run_dir(atom_symbols, boxsize, nuc_charges, positions, lambda_value, path, pp_dir, pp_type, shift2center, template_inp, template_inp_small_lambda, valence_charges)

In [107]:

compounds = ['CH4', 'H2O', 'NH3', 'HF', 'ethane', 'ethene', 'ethyne', 'HCN']
lambda_values = np.arange(0.0, 1.05, 0.05)
for comp in compounds:
    for lam in lambda_values:
        xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
        atom_symbols, nuc_charges, positions, valence_charges = pc.parse_xyz_for_CPMD_input(xyz_path)
        # define the array of lambda values
        lambda_value = np.full_like(valence_charges, fill_value=lam, dtype='float')
        # calculate correct boxsize
        num_ve = valence_charges.sum()
        pc.calculate_charge(lambda_value, num_ve, valence_charges)

In [106]:
lambda_values[3]-0.15

2.7755575615628914e-17

In [101]:
lambda_values

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

## finite difference calculations for small molecules

In [ ]:
base_lambda = np.arange(0.05, 1.05, 0.05)
compounds = ['CH4', 'NH3', 'HF', 'ethane', 'ethene', 'ethyne', 'HCN']
delta_lambda = 5e-5
paths = collections.defaultdict(list)
for comp in compounds:
    for lam in base_lambda:
        paths[comp].extend(pc.finite_difference_directories(comp, delta_lambda, lam))

In [ ]:
for c in compounds:
    uqm.write_list(f'/data/sahre/projects/finite_differences/small_molecules/{c}/run_dirs_fd', paths[c])

In [ ]:
paths['ethene']

In [ ]:
uqm.write_list('/data/sahre/projects/finite_differences/small_molecules/H2O/run_dirs_fd', paths['H2O'])

### finite differences for additional systems

In [12]:
system_paths = uqm.read_list('/data/sahre/projects/finite_differences/small_molecules/dirs')
systems = []
for s in system_paths:
    comp = s.split('/')[-2]
    lam = float(s.split('/')[-1].split('_')[-1])
    systems.append([comp,lam])

In [13]:
systems

[['ethene', 0.45]]

In [14]:
delta_lambda = 5e-5
paths = []
for comp, lam in systems:
    paths.extend(pc.finite_difference_directories(comp, delta_lambda, lam))

In [15]:
uqm.write_list('/data/sahre/projects/finite_differences/small_molecules/fd_rest',paths)

# prepare base directories for small molecules with fd

In [ ]:
compounds = ['CH4', 'H2O', 'NH3', 'HF', 'ethane', 'ethene', 'ethyne', 'HCN']
paths_tar=[]
lambda_values = np.linspace(0.05, 0.95, 10)#np.linspace(0, 0.9, 10)
for comp in compounds:
    for lam in lambda_values:
        
        # path to PP-files and type of PP (pbe)
        pp_dir = '/data/sahre/PP_LIBRARY'
        pp_type = 'GH_PBE'
        
        # input templates
        template_inp = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template.inp'
        template_inp_small_lambda = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_small_lambda.inp'
        
        # read optimized geometry and extract molecule specific parameters
        xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
        atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
        # define the array of lambda values
        lambda_value = np.full_like(valence_charges, fill_value=lam, dtype='float')
        # calculate correct boxsize
        num_ve = valence_charges.sum()
        reference_density = 8/(15**3) # density of UEG for a boxlength of 15 Ang and 8 valence electrons
        boxsize = (num_ve/reference_density)**(1/3)

        shift2center = True

        path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/lam_{np.round(lam, 3)}'
        
        # paths_tar.append(f'./{comp}/lam_{np.round(lam, 3)}\n')
        pc.create_run_dir(atom_symbols, boxsize, nuc_charges, positions, lambda_value, path, pp_dir, pp_type, shift2center, template_inp, template_inp_small_lambda, valence_charges)

In [ ]:
paths_tar
with open('/data/sahre/projects/finite_differences/small_molecules/sm2', 'w') as f:
    for p in paths_tar:
        f.write(p)

# GEOMETRY OPTIMIZATION

In [ ]:
def create_run_dir_boxsize(path, atom_symbols, boxsize, nuc_charges, positions, lambda_value, shift2center, valence_charges):

    # create directory if not exists
    os.makedirs(path, exist_ok=True)
    num_gpts = 0

    # shift molecule to center of box
    if shift2center:
        coords_final = eqd.shift2center(positions, np.array([boxsize, boxsize, boxsize])/2)
    else:
        coords_final = positions
        
    # lambda dependent quantities

    # add correct number of electrons such that system stays isoelectronic to target molecule
    num_ve = valence_charges.sum()
    charge = calculate_charge(lambda_value, num_ve, valence_charges, integer_electrons=False)

    # generate input file
    input_path = os.path.join(path, 'run.inp')
    
    # start from random wavefunction as initial guess if lambda < 0.5 otherwise the calculation will crash
    if type(lambda_value) == float:
        lam_min = lambda_value
    else:
        lam_min = np.amin(lambda_value)
        
    if lam_min > 0.501:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
    else:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

    # generate pp-files
    if type(lambda_value) == float:
        write_pp_files_compound(atom_symbols, lambda_value, path, pp_dir, pp_type)
    else:
        write_pp_files_compound_partial(atom_symbols, lambda_value, path, pp_dir, pp_type)

In [ ]:
compounds = ['CH4', 'H2O', 'NH3', 'HF', 'ethane', 'ethene', 'ethyne', 'HCN']

comp = 'HCN'

for comp in compounds:
    pp_dir = '/data/sahre/PP_LIBRARY'
    pp_type = 'GH_PBE'

    template_inp = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_geom.inp'

    xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
    atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
    lambda_value = np.full_like(valence_charges, fill_value=1.0, dtype='float')
    # calculate correct boxsize
    num_ve = valence_charges.sum()
    reference_density = 8/(15**3) # density of UEG for a boxlength of 15 Ang and 8 valence electrons
    boxsize = (num_ve/reference_density)**(1/3)
    shift2center = True

    path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}'

    create_run_dir_boxsize(path, atom_symbols, boxsize, nuc_charges, positions, lambda_value, shift2center, valence_charges)

# Boxsize convergence studies

In [ ]:
# optimize geometry (at lambda = 1)

# create input for different boxsizes
## remove MESH key and get grid points directly from PW CUTOFF

In [ ]:
def create_run_dir_boxsize(path, atom_symbols, boxsize, nuc_charges, positions, lambda_value, shift2center, valence_charges):

    # create directory if not exists
    os.makedirs(path, exist_ok=True)
    num_gpts = 0

    # shift molecule to center of box
    if shift2center:
        coords_final = eqd.shift2center(positions, np.array([boxsize, boxsize, boxsize])/2)
    else:
        coords_final = positions
        
    # lambda dependent quantities

    # add correct number of electrons such that system stays isoelectronic to target molecule
    num_ve = valence_charges.sum()
    charge = calculate_charge(lambda_value, num_ve, valence_charges, integer_electrons=False)

    # generate input file
    input_path = os.path.join(path, 'run.inp')
    
    # start from random wavefunction as initial guess if lambda < 0.5 otherwise the calculation will crash
    if type(lambda_value) == float:
        lam_min = lambda_value
    else:
        lam_min = np.amin(lambda_value)
        
    if lam_min > 0.501:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
    else:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

    # generate pp-files
    if type(lambda_value) == float:
        write_pp_files_compound(atom_symbols, lambda_value, path, pp_dir, pp_type)
    else:
        write_pp_files_compound_partial(atom_symbols, lambda_value, path, pp_dir, pp_type)

In [ ]:
comp = 'H2O'
boxsize = 20


pp_dir = '/data/sahre/PP_LIBRARY'
pp_type = 'GH_PBE'
template_inp = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template.inp'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_small_lambda.inp'

xyz_path = f'/data/sahre/projects/finite_differences/boxsize/{comp}_{boxsize}/GEOMETRY.xyz'
atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
lambda_value = np.full_like(valence_charges, fill_value=0.0, dtype='float')

shift2center = True
path = f'/data/sahre/projects/finite_differences/boxsize/{comp}_{boxsize}_lam{lambda_value[0]}'

In [ ]:
create_run_dir_boxsize(path, atom_symbols, boxsize, nuc_charges, positions, lambda_value, shift2center, valence_charges)

### UEG vs NVE

In [ ]:
# compounds = ['ethane', 'ethene', 'ethyne']
compounds = ['H2O']
boxsizes = [20, 22.5, 25]


for comp in compounds:
    for boxsize in boxsizes:
        pp_dir = '/data/sahre/PP_LIBRARY'
        pp_type = 'GH_PBE'
        template_inp = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template.inp'
        template_inp_small_lambda = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_small_lambda.inp'

        xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
        atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
        lambda_value = np.full_like(valence_charges, fill_value=0.0, dtype='float')
        num_ve = valence_charges.sum()
        shift2center = True
        path = f'/data/sahre/projects/finite_differences/boxsize/UEG/ve_{num_ve}/a_{boxsize}'
        pc.create_run_dir(atom_symbols, boxsize, nuc_charges, positions, lambda_value, path, pp_dir, pp_type, shift2center, template_inp, template_inp_small_lambda, valence_charges)

# central differences butanol

In [ ]:
def create_run_dir(path, atom_symbols, nuc_charges, positions, lambda_value, valence_charges = None, integer_electrons = False):

    # create directory if not exists
    os.makedirs(path, exist_ok=True)

    # set parameters independent of lambda value
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    # coords_final = eqd.shift2center(positions, np.array([boxsize, boxsize, boxsize])/2)
    coords_final = positions
    # lambda dependent quantities

    # add correct number of electrons such that system stays isoelectronic to target molecule\
    charge = calculate_charge(lambda_value, num_ve, valence_charges, integer_electrons)

    # generate input file
    input_path = os.path.join(path, 'run.inp')
    
    # start from random wavefunction as initial guess if lambda < 0.5 otherwise the calculation will crash
    if type(lambda_value) == float:
        lam_min = lambda_value
    else:
        lam_min = np.amin(lambda_value)
        
    if lam_min > 0.501:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
    else:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

    # generate pp-files
    if type(lambda_value) == float:
        write_pp_files_compound(atom_symbols, lambda_value, path, pp_dir, pp_type)
    else:
        write_pp_files_compound_partial(atom_symbols, lambda_value, path, pp_dir, pp_type)
        


### calculation of finite differences

In [ ]:
compound_directory = '/data/sahre/projects/atomic_force_field/bond_stretch/butanol/dist_0.0'
lambda_directories_names = ['ve_06',  've_13',  've_19',  've_26',  've_32']
base_lambda_values = np.array([6,13,19,26,32])/32
dlambda = 1e-5 # finite difference


In [ ]:
for ve, base_lambda in zip(lambda_directories_names, base_lambda_values):

    xyz_path = f'{compound_directory}/{ve}/GEOMETRY.xyz'
    atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
    
    
    for idx, derivative in enumerate(atom_symbols['elIdx']):
        
        for sign, label in zip([-1, 1], ['bw', 'fw']):
            
            path = f'{compound_directory}/{ve}/dE_d{derivative}_{label}' # path to directory where run.inp and pseudopotentials will be written to
            lambda_value = [base_lambda]*len(atom_symbols['elIdx'])
            lambda_value[idx] = lambda_value[idx] + sign*dlambda
            # print(lambda_value)
    
            atom_symbols = atom_symbols # string element + integer index
            nuc_charges = nuc_charges# list of nuclear charges
            valence_charges = valence_charges
            positions = positions# atom positions


            integer_electrons = False# rescales lambda value such that number of electrons from nuclei is an integer only kept for backwards compability

            pp_dir = '/data/sahre/PP_LIBRARY'
            pp_type = 'GH_PBE'
            template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe.inp'
            template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe_small_lambda.inp'

            create_run_dir(path, atom_symbols, nuc_charges, positions, lambda_value, valence_charges, integer_electrons)
            
            # copy RESTART.1 and LATEST from base directory
#             restart_path_source = f'{compound_directory}/{ve}/RESTART.1'
#             restart_path_dest = f'{compound_directory}/{ve}/dE_d{derivative}_{label}/RESTART.1'
#             shutil.copy(restart_path_source, restart_path_dest)
            
#             latest_path_source = f'{compound_directory}/{ve}/LATEST'
#             latest_path_dest = f'{compound_directory}/{ve}/dE_d{derivative}_{label}/LATEST'
#             shutil.copy(latest_path_source, latest_path_dest)
            
            # add restart option
            input_dir = f'{compound_directory}/{ve}/dE_d{derivative}_{label}'
            cpmd_io.fix_input_file(input_dir)

### Hellmann-Feynman derivatives

In [ ]:
for ve, base_lambda in zip(['ve_02', 've_03', 've_05', 've_06', 've_08'], np.array([2,3,5,6,8])/8):

    xyz_path = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/GEOMETRY.xyz'
    atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
    
    masks = [[1,1,1], [1,0,0], [0,1,0], [0,0,1]]
    
    for derivative, mask in zip(['lam', 'H1', 'H2', 'O3'], masks):
        for label in ['lam_r', 'lam_t']:
            lambda_value = []
            path = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}' # path to directory where run.inp and pseudopotentials will be written to
            if label == 'lam_r':
                for m in mask:
                    if m == 1:
                        lambda_value.append(0)
                    else:
                        lambda_value.append(base_lambda)
            elif label == 'lam_t':
                for m in mask:
                    if m == 1:
                        lambda_value.append(1)
                    else:
                        lambda_value.append(base_lambda)
    
            atom_symbols = atom_symbols # string element + integer index
            nuc_charges = nuc_charges# list of nuclear charges
            valence_charges = valence_charges
            positions = positions# atom positions


            integer_electrons = False# rescales lambda value such that number of electrons from nuclei is an integer only kept for backwards compability

            pp_dir = '/data/sahre/PP_LIBRARY'
            pp_type = 'GH_PBE'
            template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe.inp'
            template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe_small_lambda.inp'

            create_run_dir(path, atom_symbols, nuc_charges, positions, lambda_value, valence_charges, integer_electrons)
            
            # copy RESTART.1 and LATEST from base directory
            restart_path_source = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/RESTART.1'
            restart_path_dest = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}/RESTART.1'
            shutil.copy(restart_path_source, restart_path_dest)
            
            latest_path_source = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/LATEST'
            latest_path_dest = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}/LATEST'
            shutil.copy(latest_path_source, latest_path_dest)
            
            # add restart option
            input_dir = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}'
            cpmd_io.fix_input_file(input_dir)
            
            input_file_path = os.path.join(input_dir, 'run.inp')
            cpmd_io.modify_input_parameter(input_file_path, 'MAXITER', 1)

In [ ]:
-17.21914198-8.96085551

### prepare forward and backward calculations

In [ ]:
for ve, base_lambda in zip(['ve_02', 've_03', 've_05', 've_06'], np.array([2,3,5,6])/8):

    xyz_path = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/GEOMETRY.xyz'
    atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
    
    dlambda = 1e-5 # finite difference
    increments = np.array([[dlambda, dlambda, dlambda], [dlambda, 0, 0], [0,dlambda,0], [0,0,dlambda]])
    
    for derivative, lambdas in zip(['lam', 'H1', 'H2', 'O3'], increments):
        for sign, label in zip([-1, 1], ['bw', 'fw']):
            path = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}' # path to directory where run.inp and pseudopotentials will be written to
            lambda_value = sign*lambdas + base_lambda# lambda value can be a single value (for full derivative) or list with len == number of atoms for partial derivatives
            # print(lambda_value)
    
            atom_symbols = atom_symbols # string element + integer index
            nuc_charges = nuc_charges# list of nuclear charges
            valence_charges = valence_charges
            positions = positions# atom positions


            integer_electrons = False# rescales lambda value such that number of electrons from nuclei is an integer only kept for backwards compability

            pp_dir = '/data/sahre/PP_LIBRARY'
            pp_type = 'GH_PBE'
            template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe.inp'
            template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe_small_lambda.inp'

            create_run_dir(path, atom_symbols, nuc_charges, positions, lambda_value, valence_charges, integer_electrons)
            
            # copy RESTART.1 and LATEST from base directory
            restart_path_source = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/RESTART.1'
            restart_path_dest = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}/RESTART.1'
            shutil.copy(restart_path_source, restart_path_dest)
            
            latest_path_source = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/LATEST'
            latest_path_dest = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}/LATEST'
            shutil.copy(latest_path_source, latest_path_dest)
            
            # add restart option
            input_dir = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}'
            cpmd_io.fix_input_file(input_dir)

### for lambda value paths

In [ ]:
def wrapper_ase2(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda):
    """
    generates all necessary files for a cpmd calculation using an atoms object from ase as input
    """
    # calculation parameters (independent of lambda value)
    atom_symbols = atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve/2) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    coords_final = eqd.shift2center(atoms.get_positions(), np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val in lambda_values:
        #print(lam_val)
        elec_pp = np.round(lam_val*num_ve) # electrons coming from pseudopotentials
        charge = elec_pp-num_ve # electrons added to conserve total number of electrons

        lambda_path = os.path.join(compound_path, f'lam_{lam_val}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if lam_val > 0.501:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
        else:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

        # generate pp-files
        write_pp_files_compound(atom_symbols, lam_val, lambda_path, pp_dir, pp_type)
        
        print(lambda_path)
        
def wrapper_ase3(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda):
    """
    generates all necessary files for a cpmd calculation using an atoms object from ase as input
    """
    # calculation parameters (independent of lambda value)
    atom_symbols = ['H1', 'H2']#atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve/2) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    coords_final = eqd.shift2center(atoms.get_positions(), np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val1, lam_val2 in lambda_values:
        #print(lam_val)
        elec_pp = int(np.round(lam_val1 + lam_val2)) # electrons coming from pseudopotentials
        charge = elec_pp-num_ve # electrons added to conserve total number of electrons
        lambda_path = os.path.join(compound_path, f'H2_lam_{lam_val2}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if lam_val1 > 0.501:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
        else:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

        # generate pp-files
        #write_pp_files_compound(atom_symbols, lam_val, lambda_path, pp_dir, pp_type)
        write_pp_files_compound_partial(atom_symbols, [lam_val1, lam_val2], lambda_path, pp_dir, pp_type)
        print(lambda_path)

In [ ]:
# define parameters
pp_dir = '/data/sahre/PP_LIBRARY'
pp_type = 'SG_LDA'
template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_cutoff.inp'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_small_lambda_cutoff.inp'
compound_path = '/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched'
dlam = 0.00001
#lambda_values = np.array([0.11, 0.31, 0.41, 0.51, 0.71, 0.91]) + dlam

lambda_values = np.sort(np.concatenate((np.arange(0.0, 1.05, 0.05), np.arange(0.0, 1.05, 0.05)+dlam)))

atoms = aio.read('/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched.xyz')

wrapper_ase2(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda)

In [ ]:
# define parameters
pp_dir = '/data/sahre/PP_LIBRARY'
pp_type = 'SG_LDA'
template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_cutoff.inp'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_small_lambda_cutoff.inp'
compound_path = '/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/'
dlam = 0.00001

lam = np.arange(0.0, 1.05, 0.05)

lambda_values = np.array([lam, lam+dlam]).T

atoms = aio.read('/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched.xyz')

wrapper_ase3(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda)

In [ ]:
np.arange(0.0, 1.05, 0.05)